In [ ]:
# Set up API key and do the necessary imports
import os
from taskgen import *

os.environ['OPENAI_API_KEY'] = ""

## DATA LOADING

In [ ]:
import json
with open('selected_problems_level_5.json', 'r') as file:
    selected_problems = json.load(file)

## TaskGen Agent And Functions 

In [ ]:

def python_run_tool(code_snippet: str) -> str:
    '''Runs code_snippet and outputs the result of all print statements'''
    import sys
    import io
    import math
    import numpy
    import random
    import datetime
    import re
    import matplotlib
    import pandas
    import sympy
    import plotly
    import numpy as np
    # Disable file access
    def restricted_open(*args, **kwargs):
        raise PermissionError("File access is restricted")
    
    # Capture the output
    old_stdout = sys.stdout
    sys.stdout = io.StringIO()

    try:
        # Safe environment to execute the user code
        allowed_globals = {
            '__builtins__': {
                'print': print,
                'range': range,
                'len': len,
                'int': int,
                'float': float,
                'str': str,
                'list': list,
                'dict': dict,
                'set': set,
                'tuple': tuple,
                'abs': abs,
                'min': min,
                'max': max,
                'sum': sum,
                'any': any,
                'all': all,
                'sorted': sorted,
                'zip': zip,
                'map': map,
                'filter': filter,
                '__import__': __import__,
                'math': math,  # Allow access to the math module
                'datetime': datetime, # Allow access to datetime module
                'random': random, # Allow access to random module
                'numpy': numpy, # Allow access to numpy module
                're': re,
                'pandas': pandas,
                'open': restricted_open,  # Override open to restrict file access
                'sympy': sympy,                
            }
        }

        safe_locals = {}

        exec(code_snippet, allowed_globals, safe_locals)
        output = sys.stdout.getvalue()
    except Exception as e:
        output = f"Error: {e}"
    finally:
        # Restore the original stdout
        sys.stdout = old_stdout

    return output

python_generator_tool = Function('''Generate code based only on <instruction: str> without additional context.
    Ensure that you define all variables and list out all imports.
    You can only import the following modules: math, numpy, random, datetime, re, matplotlib, pandas, sympy, plotly
    ***Do not use short forms for modules for example use import numpy instead of import numpy as np***
    Do not define any functions
    You are not able to use the Equipped Functions using this tool.
    Ensure all required output are in print statements''',
                                     output_format = {'Output Code': 'type: code'}, fn_name = 'python_generator_tool',model="gpt-4o")



python_debug_tool = Function('''Debugs Python Code and returns corrected code.
Instruction: <instruction: str>
Current Code: <python_code: str>
Error Message: <error_msg: str>''',
                                 output_format = {'Thoughts': 'How to correct code', 'Corrected Code': 'type: code'}, fn_name = 'python_debug_tool',model="gpt-4o")




# Uses LLM to generate Code
def python_generate_and_run_code_tool(shared_variables, instruction: str) -> str:
    ''' Generates and runs code based on instruction. Returns 1) the result of all print statements in code, or error messages, and 2) the code '''
    # from termcolor import colored
    
    # Append context to tool
    if shared_variables and 'agent' in shared_variables:
        instruction = f"Context: {shared_variables['agent'].overall_task}\nPrevious Subtasks: {shared_variables['agent'].subtasks_completed}\nInstruction: {instruction}"
    # Generate Code
    python_code = python_generator_tool(instruction)['Output Code']
    
    # Run and Debug Code
    for _ in range(3):
        output = python_run_tool(python_code)

        if output[:5] == "Error":
            debugged_code = python_debug_tool(instruction, python_code, output)
            python_code = debugged_code['Corrected Code']

        else:
            break
            
    return output, python_code

In [ ]:
# Agent for Math Problem Solver

agent = Agent('Math Problem Solver', 
'''You are a math expert skilled in solving a wide range of math problems, including symbolic, algebraic, and numerical challenges. Your task is to analyze the given problem, break it down into manageable steps, and provide a comprehensive solution with clear, logical reasoning. Ensure that your explanations are thorough, using relevant mathematical principles and terminology. Importantly, present the final answer clearly and explicitly, such that no further calculations are required from the user. Please note: do not use any large language models to derive the solution, you are free to use the tools given to you for the solution like sympy library; all steps should be manually explainable and grounded in mathematical theory. When using code to solve the problem, only print out the final answer. You should only use code to solve the problem if it is necessary and should not be the primary method of solving the problem.
''',model="gpt-4o").assign_functions(
    [python_generate_and_run_code_tool])


# Assign the shared variables for agent so code tool can get context of the task
agent.shared_variables['agent'] = agent


### Example : Counting and Probability

In [ ]:
example_prob = selected_problems['counting_and_probability'][0]

with open(example_prob, 'r') as file:
    example_data = json.load(file)

example_data["problem"]

#### Simple example of running the agent on the problem

In [ ]:
agent.reset()    
soln = agent.run(example_data["problem"])